<a href="https://colab.research.google.com/github/Buy-me-some-str/practice/blob/main/radar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch

In [127]:
distances1 = torch.tensor([-5, 1, 11, 0.7])
distances2 = torch.tensor([-3, 7, 4, 3])
radar_coordinates = torch.tensor([[0, 0, 0],
                                  [1, 0, 0],
                                  [1, 1, 1],
                                  [1, 1, 0]])
point = torch.tensor([2, 2, 2])
distances1 = torch.tensor([(2-0)**2+(2-0)**2+(2-0)**2,
                           (2-1)**2+(2-0)**2+(2-0)**2,
                           (2-1)**2+(2-1)**2+(2-1)**2,
                           (2-1)**2+(2-1)**2+(2-0)**2])
distances1

tensor([12,  9,  3,  6])

In [157]:
def point_coordinates(distances: torch.Tensor, 
                      radar_coordinates: torch.Tensor)->torch.Tensor:
    A = torch.ones(size = (3, 3))
    b = torch.ones(size = (3, 1))
    for i in range(0, 3):
        A[i] = radar_coordinates[i+1]-radar_coordinates[i]
        b[i] = (distances[i]**2 - distances[i+1]**2 + sum(radar_coordinates[i]**2) - sum(radar_coordinates[i+1]**2))/2
        #print('sum(radar_coordinates[',i+1,']**2)',sum(radar_coordinates[i+1]**2))
        print(distances[i])
    #print("sum",sum(radar_coordinates[i]**2), radar_coordinates[i], i)
    print(A)
    print(b)
    x = torch.linalg.solve(A, b)
    return x.squeeze(dim=1)
point_coordinates(distances1, radar_coordinates)

tensor(12)
tensor(9)
tensor(3)
tensor([[ 1.,  0.,  0.],
        [ 0.,  1.,  1.],
        [ 0.,  0., -1.]])
tensor([[ 31.],
        [ 35.],
        [-13.]])


tensor([31., 22., 13.])

In [131]:
def checking_coordinates(point:torch.Tensor,distances:torch.Tensor, radar_coordinates:torch.Tensor)->torch.bool:
    A = torch.ones(size=(radar_coordinates.shape[0], 1))
    for i in range(A.shape[0]):
        for j in range(len(point)):
            A[i]+=(point[j]-radar_coordinates[i][j])**2
            #print("(",point[j],"-",radar_coordinates[i][j],")**2")
        #print("\n")
    flag = True
    for i in range(A.shape[0]):
        flag = False if A[i]!=distances[i]**2 else True
    return flag

In [132]:
checking_coordinates(point1, distances1, radar_coordinates)

False

In [135]:
point1 = point_coordinates(distances=distances1, 
                          radar_coordinates=radar_coordinates)
point2 = point_coordinates(distances=distances2,
                          radar_coordinates=radar_coordinates)
points = torch.stack((point1, point2))
distances = torch.stack((distances1, distances2))
points, distances

(tensor([[ 32.0000,  23.0000,  14.0000],
         [-19.5000,  20.5000,  -3.0000]]), tensor([[12,  9,  3,  6],
         [-3,  7,  4,  3]]))

In [136]:
def selection_coordinates(distances:torch.Tensor,radar_coordinates:torch.Tensor)->torch.Tensor:
    answer = []
    iterator = torch.tensor([[0, 0, 0, 0],
                             [0, 0, 0, 1],
                             [0, 0, 1, 0],
                             [0, 0, 1, 1],
                             [0, 1, 0, 0],
                             [0, 1, 0, 1],
                             [0, 1, 1, 0],
                             [0, 1, 1, 1],
                             [1, 0, 0, 0],
                             [1, 0, 0, 1],
                             [1, 0, 1, 0],
                             [1, 0, 1, 1],
                             [1, 1, 0, 0],
                             [1, 1, 0, 1],
                             [1, 1, 1, 0],
                             [1, 1, 1, 1]])
    for i in range(iterator.shape[0]):
        test_distances = []
        for j in range(iterator.shape[1]):
            test_distances.append(distances[iterator[i][j]][j])
        test_distances = torch.tensor(test_distances)
        test_point = point_coordinates(distances=test_distances,
                                       radar_coordinates=radar_coordinates)
        if checking_coordinates(point=test_point,
                                distances=test_distances,
                                radar_coordinates=radar_coordinates):
            answer.append(test_distances)
    return torch.tensor(answer)
selection_coordinates(distances, radar_coordinates)

tensor([])